In [0]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}
!mkdir -p drive
!google-drive-ocamlfuse drive



In [5]:
!pip install tqdm

    100% |████████████████████████████████| 51kB 2.5MB/s 


In [2]:
import os
os.chdir('drive/aman')
os.listdir()

['Copy of mnist.ipynb',
 'predictions_on_test.csv',
 'ngrok-stable-linux-amd64.zip',
 'Test',
 'model-013.h5',
 'model-012.h5',
 'model-011.h5',
 'model-010.h5',
 'model-009.h5',
 'model-003.h5',
 'model-002.h5',
 'model-001.h5',
 '.logs',
 'main_model.h5',
 'checkpoints',
 'logs',
 'pos',
 'Negs',
 'ngrok']

In [3]:
'''Trains a simple convnet on the MNIST dataset.
Gets to 99.25% test accuracy after 12 epochs
(there is still a lot of margin for parameter tuning).
16 seconds per epoch on a GRID K520 GPU.
'''

from __future__ import print_function
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K

batch_size = 128
num_classes = 10
epochs = 20

# input image dimensions
img_rows, img_cols = 28, 28

# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu', name = 'dense1'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])
# define callback 
check_point = keras.callbacks.ModelCheckpoint('./Checkpoint',
                                              monitor='val_loss',
                                              verbose=0,
                                              save_best_only=True,
                                              save_weights_only=False,
                                              mode='auto', period=1)

early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss',
                                               min_delta=0, patience=4,
                                               verbose=1,
                                               mode='auto',
                                               )
# tensor_board = keras.callbacks.Tensorboard('drive/Colab Notebooks/logs')

#training the model
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=12,
          verbose=1,
          validation_data=(x_test, y_test),
          callbacks = [early_stopping])

score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Using TensorFlow backend.


11493376/11490434 [==============================] - 1s 0us/step
x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples
Train on 60000 samples, validate on 10000 samples
Epoch 1/12
60000/60000 [==============================] - 14s 231us/step - loss: 0.2726 - acc: 0.9166 - val_loss: 0.0582 - val_acc: 0.9804
Epoch 2/12
22400/60000 [==========>...................] - ETA: 6s - loss: 0.0929 - acc: 0.9709

60000/60000 [==============================] - 11s 190us/step - loss: 0.0892 - acc: 0.9731 - val_loss: 0.0425 - val_acc: 0.9856
Epoch 3/12
60000/60000 [==============================] - 11s 188us/step - loss: 0.0644 - acc: 0.9809 - val_loss: 0.0462 - val_acc: 0.9845
Epoch 4/12
47616/60000 [======================>.......] - ETA: 2s - loss: 0.0562 - acc: 0.9835

60000/60000 [==============================] - 11s 190us/step - loss: 0.0538 - acc: 0.9842 - val_loss: 0.0320 - val_acc: 0.9889
Epoch 5/12
60000/60000 [==============================] - 12s 194us/step - loss: 0.0464 - acc: 0.9862 - val_loss: 0.0334 - val_acc: 0.9891
Epoch 6/12
54912/60000 [==========================>...] - ETA: 0s - loss: 0.0412 - acc: 0.9874

60000/60000 [==============================] - 11s 190us/step - loss: 0.0411 - acc: 0.9873 - val_loss: 0.0261 - val_acc: 0.9920
Epoch 7/12
60000/60000 [==============================] - 11s 183us/step - loss: 0.0354 - acc: 0.9889 - val_loss: 0.0257 - val_acc: 0.9914
Epoch 8/12
60000/60000 [==============================] - 11s 179us/step - loss: 0.0338 - acc: 0.9893 - val_loss: 0.0253 - val_acc: 0.9913
Epoch 9/12
  768/60000 [..............................] - ETA: 11s - loss: 0.0336 - acc: 0.9883

60000/60000 [==============================] - 11s 178us/step - loss: 0.0291 - acc: 0.9909 - val_loss: 0.0255 - val_acc: 0.9927
Epoch 10/12
60000/60000 [==============================] - 11s 177us/step - loss: 0.0270 - acc: 0.9915 - val_loss: 0.0267 - val_acc: 0.9919
Epoch 11/12
48768/60000 [=======================>......] - ETA: 1s - loss: 0.0269 - acc: 0.9915

60000/60000 [==============================] - 11s 181us/step - loss: 0.0267 - acc: 0.9916 - val_loss: 0.0250 - val_acc: 0.9922
Epoch 12/12
60000/60000 [==============================] - 11s 178us/step - loss: 0.0236 - acc: 0.9928 - val_loss: 0.0240 - val_acc: 0.9930
Test loss: 0.0239709717479971
Test accuracy: 0.993


In [20]:
#data_load

import os 
import cv2
from tqdm import tqdm


def load_data(pos_path, neg_path, val_split):
  length = len(os.listdir(pos_path)) + len(os.listdir(neg_path))
  images  = np.empty([length, 28,28])
  labels = np.empty ([length,2])
  count = 0
  for img in (os.listdir(pos_path)):
    image = cv2.imread(os.path.join(pos_path, img))[:,:,0]
    images[count] = image
    labels[count][0] = 1
    labels[count][1] = 0
    count += 1
    if count / 50 == 0 : print(f'done : {count}')
  print('positives done!')
  for img in (os.listdir(neg_path)):
    image = cv2.imread(os.path.join(neg_path, img))[:,:,0]
    images[count] = image
    labels[count][1] = 1
    labels[count][0] = 0
    count += 1
    if count == length:
      print('negatives done!')
      break
  
  #loop for randomizing
  rimages  = np.empty([length, 28,28])
  rlabels = np.empty ([length,2])
  count = 0 
  for i in np.random.permutation(length):
    rimages[i] = images[count]
    rlabels[i]= labels[count]
    count += 1
  print('randomized')
  split = np.int_((1- val_split)*length)
  rimages = np.reshape(rimages, [length, 28, 28, 1])
  X_train = rimages[:split] 
  X_test = rimages[split:]
  Y_train = rlabels[:split]
  Y_test = rlabels[split:]
  return X_train, Y_train, X_test , Y_test

In [21]:
X_train, Y_train, X_validation , Y_validation = load_data('pos', 'Negs', val_split= 0.2)
print(X_train.shape, Y_train.shape, X_validation.shape , Y_validation.shape)


positives done!
negatives done!
randomized
(1444, 28, 28, 1) (1444, 2) (361, 28, 28, 1) (361, 2)


In [22]:
from keras.layers import Input
from keras.models import Model
import numpy as np
import pandas as pd

In [27]:
# model.save('mnist.h5')
inputs = Input([28,28,1])

mnist = Model(inputs = model.layers[0].input,  outputs= model.get_layer('dense1').output)

mnist_output = mnist(inputs)
# dropout = Dropout(0.5)(mnist_output)



dense = Dense(2, activation = 'softmax')(mnist_output)

main_model = Model(inputs = inputs , outputs = dense, name = 'aman_proj')


In [28]:
main_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         (None, 28, 28, 1)         0         
_________________________________________________________________
model_4 (Model)              (None, 128)               1198592   
_________________________________________________________________
dense_8 (Dense)              (None, 2)                 258       
Total params: 1,198,850
Trainable params: 1,198,850
Non-trainable params: 0
_________________________________________________________________


In [31]:
main_model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

In [0]:
main_model.fit(X_train, Y_train, validation_data = [X_validation, Y_validation], shuffle = True, epochs = 100)

In [186]:
main_model.save('main_model1.h5')


In [12]:
from keras.models import load_model

prediction = load_model('main_model.h5')
import cv2

In [7]:
#test loading

def load_test(path):
  x = np.empty([len(os.listdir(path)), 28,28,1])
  count = 0 
  for file in os.listdir(path):
    img = cv2.imread(os.path.join(path, file))[:,:,0]
    img = img.reshape([28,28,1])
    x[count] = img
    count += 1
    if count == len(os.listdir(path)): break
  return x 

In [11]:
import numpy as np
x = load_test('Test')
print('done loading')

In [22]:
prob = prediction.predict(x) 


In [23]:
print(prob)

[[0.0000000e+00 1.0000000e+00]
 [1.1268439e-04 9.9988735e-01]
 [0.0000000e+00 1.0000000e+00]
 [0.0000000e+00 1.0000000e+00]
 [0.0000000e+00 1.0000000e+00]
 [1.1268439e-04 9.9988735e-01]
 [0.0000000e+00 1.0000000e+00]
 [0.0000000e+00 1.0000000e+00]
 [1.1268439e-04 9.9988735e-01]
 [0.0000000e+00 1.0000000e+00]
 [0.0000000e+00 1.0000000e+00]
 [1.1268439e-04 9.9988735e-01]
 [1.1268439e-04 9.9988735e-01]
 [0.0000000e+00 1.0000000e+00]
 [0.0000000e+00 1.0000000e+00]
 [0.0000000e+00 1.0000000e+00]
 [0.0000000e+00 1.0000000e+00]
 [0.0000000e+00 1.0000000e+00]
 [0.0000000e+00 1.0000000e+00]
 [0.0000000e+00 1.0000000e+00]
 [0.0000000e+00 1.0000000e+00]
 [0.0000000e+00 1.0000000e+00]
 [0.0000000e+00 1.0000000e+00]
 [1.1268439e-04 9.9988735e-01]
 [1.1268439e-04 9.9988735e-01]
 [1.1268439e-04 9.9988735e-01]
 [0.0000000e+00 1.0000000e+00]
 [0.0000000e+00 1.0000000e+00]
 [0.0000000e+00 1.0000000e+00]
 [0.0000000e+00 1.0000000e+00]
 [0.0000000e+00 1.0000000e+00]
 [0.0000000e+00 1.0000000e+00]
 [1.1268

In [41]:
df = pd.DataFrame(prob, columns = ['positive', 'negative'])
df['filenames'] = os.listdir('Test')

In [42]:
df.head()

,positive,negative,filenames
0,0.000000,1.000000,5990.JPG
1,0.000113,0.999887,3380.JPG
2,0.000000,1.000000,2924.JPG
3,0.000000,1.000000,9951.JPG
4,0.000000,1.000000,6325.JPG


In [43]:
df.to_csv('predictions_on_test.csv')